# FalkonHEP: Density Ratio plot


## Import

In [24]:

import numpy as np
import torch

import matplotlib.pyplot as plt
from falkonhep import LogFalkonHEPModel
from falkonhep.utils import normalize_features



## Experiment Parameters

In [25]:
R = 200000 # reference size
B = 3000 # background expected size
S = 0 # signal expected size


reference_path = "" # directory containing reference files
data_path = "" # directory containing data files
out_path = "" # output directory

features = [] # features used in model training
prj_feature = '' # (generally high-level) feature

normalize = True # if (training) data has to be normalized
sig_type = 2 # (0: no signal, 1: separed, 2: mixed)
weight = B / R

bins = np.linspace(-200, 600, 10) #np.array([]) # how distributions are binned (specify the bin edges)

## Model Parameters

In [26]:
params = {
    'sigma' : 3.0, # length-scale of the Gaussian kernel
    'penalty_list' : [1e-7], # list of regularization parameters
    'iter_list' : [1000000], # list of maximum number of CG iterations
    'M' : 10000, # number of Nystrom centers
    'keops_active': "auto", # if pykeops is used to speed-up computations (optional, default "no")
    'seed' : 12, # seed for Nystrom centers selection
    'cg_tol' : np.sqrt(1e-7), # CG tolerance
    'use_cpu' : False # if falkon will be executed in cpu (optional, default False)
}

# Load data

In [27]:

model = LogFalkonHEPModel(reference_path, data_path, out_path)

# seeds for reproducibility
ref_seed = 12 
data_seed = 13

# Load data (non normalized): it will load both training and test features
reference_set, data_set, bck_size, sig_size = model.generate_dataset(R, B, S, np.concatenate([features, [prj_feature]]), 
                                                                        normalize=False, 
                                                                        sig_type=sig_type,
                                                                        cut=None, 
                                                                        ref_seed=ref_seed, 
                                                                        sig_seed=data_seed
                                                                        )

# Separe training reference/data 
reference_training, data_training = reference_set[:,:-1], data_set[:,:-1] 
if normalize:
    reference_training, data_training = normalize_features(reference_set[:,:-1], data_set[:,:-1])
reference_test, data_test = reference_set[:,-1], data_set[:,-1]

print("[--] Training/test data prepared!")

[--] Training/test data prepared!


## Fit model and make reference predictions

In [28]:
# Instantiate Falkon/LogFalkon model
model.build_model(params,  weight)

# Build training set 
Xtr = torch.from_numpy(np.vstack((reference_training, data_training)))
ytr = torch.from_numpy(model.create_labels(reference_training.shape[0], data_training.shape[0]))

# Fit the model
model.fit(Xtr, ytr)

# Make reference predictions
ref_preds = np.squeeze(model.predict(reference_training))


Iteration 0 - penalty 1.000000e-07 - sub-iterations 1000000
Stopping conjugate gradient descent at iteration 6. Solution has converged.


## Build histograms and plot result

In [29]:

# Build histograms 
ref_weight = np.ones(reference_test.shape[0]) * (B/ R)
ret_weight = np.exp(ref_preds) * (B / R)
toy_weight = np.ones(data_test.shape[0]) 

ref_dd, _ = np.histogram(reference_test,  bins=bins, weights = ref_weight)
pred_dd, _ = np.histogram(reference_test, bins=bins, weights = ret_weight)
toy_dd, _ = np.histogram(data_test, bins=bins, weights = toy_weight)

lr_toy = toy_dd / ref_dd 
lr_pred = pred_dd / ref_dd

x = (bins[1:] + bins[:-1])/2

# Plot and save results
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(x, lr_toy, 'o-', color='tomato', linewidth=2, label="Toy")
ax.plot(x, lr_pred, 'o-' ,color='royalblue', linewidth=2, label="Learned")
ax.set_title(r'Signal reconstruction', fontsize=20)
ax.set_xlabel(r'$\mathregular{t}$', fontsize=20)
ax.set_ylabel(r'$\mathregular{n(x|1)/n(x|0)}$', fontsize=20)
ax.set_ylim(bottom=-1e-1, top=13) #set y lims to better observe results
ax.legend(loc="best", fontsize=14)

plt.savefig(out_path + "/dratio_{}_{}.pdf".format(ref_seed, data_seed), transparent=True, bbox_inches="tight")
plt.close(fig)
